# Basic BeautifulSoup Demo
modified from [eholowko](https://github.com/eholowko/bootcamp-jupyter-web_scraping/blob/master/Weather_in_San_Francisco.ipynb)

**Goal**: Scrape weather.gov to collect the "Extended Forecast" information:

- Day: ```period```
- Short Description: ```short_desc```
- Full Description: ```desc```
- Temperature in Fahrenheit: ```temp_fahrenheit```

## Tasks to Achive the Goal

1. Get the "Extended Forecast" from weather.gov for Blacksburg, VA.
2. Isolate the "Extended Forecast" information
3. Clean the data into the above variables.
4. Output as a CSV


## Import code libs

In [103]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime, timedelta

## "Get" a requested web page

In [104]:
bb_page = requests.get("https://forecast.weather.gov/MapClick.php?lat=37.2265&lon=-80.4109")

## Parse the request's ```Response``` object ```bb_page```

Objects include multiple properties that you can access. So, the ```requests.get()``` assigned a [```Response```](https://requests.readthedocs.io/en/latest/api/#requests.Response) object to the ariable ```bb_page```. 

You can use the following 2 properties to access the HTML differently:

1. ```bb_page.text``` returns a Unicode ```string```.
2. ```bb_page.content``` returns the HTML as ```bytes```

We will use the ```bb_page.content``` below to tell BS to help us parse the "soup" of HTML.

(Hey, I didn't name these things. lol)

In [105]:
soup = BeautifulSoup(bb_page.content, 'html.parser')

Use the ```.find``` method to isolate single elements. In thise case, using an ```id``` attribute with a particular value of interest.

In [106]:
extended_forecast = soup.find(id="seven-day-forecast")
print(extended_forecast.prettify())

<div class="panel panel-default" id="seven-day-forecast">
 <div class="panel-heading">
  <b>
   Extended Forecast for
  </b>
  <h2 class="panel-title">
   Blacksburg VA
  </h2>
 </div>
 <div class="panel-body" id="seven-day-forecast-body">
  <div id="seven-day-forecast-container">
   <ul class="list-unstyled" id="seven-day-forecast-list">
    <li class="forecast-tombstone">
     <div class="tombstone-container">
      <p class="period-name">
       Veterans
       <br/>
       Day
      </p>
      <p>
       <img alt="Veterans Day: Showers, with thunderstorms also possible after 10am. Some of the storms could produce heavy rainfall.  High near 69. Southeast wind around 10 mph.  Chance of precipitation is 100%. New rainfall amounts between three quarters and one inch possible. " class="forecast-icon" src="newimages/medium/shra100.png" title="Veterans Day: Showers, with thunderstorms also possible after 10am. Some of the storms could produce heavy rainfall.  High near 69. Southeast wind 

Use the ```.find_all``` method to get ALL of the elements with the ```class``` attribute of 'tombstone-cotnainer'

In [107]:
forecast_items = extended_forecast.find_all(class_="tombstone-container")

print(forecast_items)

[<div class="tombstone-container">
<p class="period-name">Veterans<br/>Day</p>
<p><img alt="Veterans Day: Showers, with thunderstorms also possible after 10am. Some of the storms could produce heavy rainfall.  High near 69. Southeast wind around 10 mph.  Chance of precipitation is 100%. New rainfall amounts between three quarters and one inch possible. " class="forecast-icon" src="newimages/medium/shra100.png" title="Veterans Day: Showers, with thunderstorms also possible after 10am. Some of the storms could produce heavy rainfall.  High near 69. Southeast wind around 10 mph.  Chance of precipitation is 100%. New rainfall amounts between three quarters and one inch possible. "/></p><p class="short-desc">Heavy Rain</p><p class="temp temp-high">High: 69 °F</p></div>, <div class="tombstone-container">
<p class="period-name">Tonight<br/><br/></p>
<p><img alt="Tonight: Showers and possibly a thunderstorm before midnight, then a slight chance of showers between midnight and 3am.  Low around 

Check the data to ensure it is what you want

In [108]:
today = forecast_items[0]
print(today.prettify())

<div class="tombstone-container">
 <p class="period-name">
  Veterans
  <br/>
  Day
 </p>
 <p>
  <img alt="Veterans Day: Showers, with thunderstorms also possible after 10am. Some of the storms could produce heavy rainfall.  High near 69. Southeast wind around 10 mph.  Chance of precipitation is 100%. New rainfall amounts between three quarters and one inch possible. " class="forecast-icon" src="newimages/medium/shra100.png" title="Veterans Day: Showers, with thunderstorms also possible after 10am. Some of the storms could produce heavy rainfall.  High near 69. Southeast wind around 10 mph.  Chance of precipitation is 100%. New rainfall amounts between three quarters and one inch possible. "/>
 </p>
 <p class="short-desc">
  Heavy Rain
 </p>
 <p class="temp temp-high">
  High: 69 °F
 </p>
</div>


In [109]:
period = today.find(class_="period-name").get_text()
short_desc = today.find(class_="short-desc").get_text()
temp = today.find(class_="temp").get_text()

print(period)
print(short_desc)
print(temp)

VeteransDay
Heavy Rain
High: 69 °F


In [110]:
img = today.find("img")

desc = img["title"]
print(desc)

Veterans Day: Showers, with thunderstorms also possible after 10am. Some of the storms could produce heavy rainfall.  High near 69. Southeast wind around 10 mph.  Chance of precipitation is 100%. New rainfall amounts between three quarters and one inch possible. 


In [111]:
period_tags = extended_forecast.select(".tombstone-container .period-name")
periods = [pt.get_text() for pt in period_tags]
periods

['VeteransDay',
 'Tonight',
 'Saturday',
 'SaturdayNight',
 'Sunday',
 'SundayNight',
 'Monday',
 'MondayNight',
 'Tuesday']

In [112]:
short_desc = [sd.get_text() for sd in extended_forecast.select(".tombstone-container .short-desc")]
temps = [t.get_text() for t in extended_forecast.select(".tombstone-container .temp")]
desc = [d["title"] for d in extended_forecast.select(".tombstone-container img")]
print(short_desc)
print(temps)
print(desc)

['Heavy Rain', 'Showers thenSlight ChanceShowers', 'ChanceShowers', 'Slight ChanceShowers', 'Mostly Sunny', 'Mostly Clear', 'Mostly Sunny', 'Partly Cloudy', 'ChanceRain/Snowthen ChanceRain']
['High: 69 °F', 'Low: 53 °F', 'High: 57 °F', 'Low: 32 °F', 'High: 41 °F', 'Low: 24 °F', 'High: 45 °F', 'Low: 26 °F', 'High: 41 °F']
['Veterans Day: Showers, with thunderstorms also possible after 10am. Some of the storms could produce heavy rainfall.  High near 69. Southeast wind around 10 mph.  Chance of precipitation is 100%. New rainfall amounts between three quarters and one inch possible. ', 'Tonight: Showers and possibly a thunderstorm before midnight, then a slight chance of showers between midnight and 3am.  Low around 53. South wind 7 to 13 mph becoming west after midnight. Winds could gust as high as 26 mph.  Chance of precipitation is 100%. New precipitation amounts between a quarter and half of an inch possible. ', 'Saturday: A chance of showers, mainly after 11am.  Partly sunny, with a

In [113]:
weather = pd.DataFrame({"period":periods,
                       "short_desc":short_desc,
                       "temp":temps,
                       "desc":desc})
weather

,period,short_desc,temp,desc
0,VeteransDay,Heavy Rain,High: 69 °F,"Veterans Day: Showers, with thunderstorms also..."
1,Tonight,Showers thenSlight ChanceShowers,Low: 53 °F,Tonight: Showers and possibly a thunderstorm b...
2,Saturday,ChanceShowers,High: 57 °F,"Saturday: A chance of showers, mainly after 11..."
3,SaturdayNight,Slight ChanceShowers,Low: 32 °F,Saturday Night: A slight chance of showers bef...
4,Sunday,Mostly Sunny,High: 41 °F,"Sunday: Mostly sunny, with a high near 41. Nor..."
5,SundayNight,Mostly Clear,Low: 24 °F,"Sunday Night: Mostly clear, with a low around 24."
6,Monday,Mostly Sunny,High: 45 °F,"Monday: Mostly sunny, with a high near 45."
7,MondayNight,Partly Cloudy,Low: 26 °F,"Monday Night: Partly cloudy, with a low around..."
8,Tuesday,ChanceRain/Snowthen ChanceRain,High: 41 °F,"Tuesday: A chance of snow before 10am, then a ..."


In [114]:
temp_nums = weather["temp"].str.extract("(\d+)",expand=False)
weather["temp_fahrenheit"] = temp_nums.astype("int")
temp_nums

0    69
1    53
2    57
3    32
4    41
5    24
6    45
7    26
8    41
Name: temp, dtype: object

In [115]:
weather = weather.drop(["temp"],axis=1)

In [116]:
weather

,period,short_desc,desc,temp_fahrenheit
0,VeteransDay,Heavy Rain,"Veterans Day: Showers, with thunderstorms also...",69
1,Tonight,Showers thenSlight ChanceShowers,Tonight: Showers and possibly a thunderstorm b...,53
2,Saturday,ChanceShowers,"Saturday: A chance of showers, mainly after 11...",57
3,SaturdayNight,Slight ChanceShowers,Saturday Night: A slight chance of showers bef...,32
4,Sunday,Mostly Sunny,"Sunday: Mostly sunny, with a high near 41. Nor...",41
5,SundayNight,Mostly Clear,"Sunday Night: Mostly clear, with a low around 24.",24
6,Monday,Mostly Sunny,"Monday: Mostly sunny, with a high near 45.",45
7,MondayNight,Partly Cloudy,"Monday Night: Partly cloudy, with a low around...",26
8,Tuesday,ChanceRain/Snowthen ChanceRain,"Tuesday: A chance of snow before 10am, then a ...",41


In [117]:
weather["desc"] = weather["desc"].str.replace("([\w]*[\s]*[\w]*:)([\w]*)","")

/var/folders/_8/xjd7q1wx0gj4s8cqk0kd4ch00000gp/T/ipykernel_43771/3171008616.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  weather["desc"] = weather["desc"].str.replace("([\w]*[\s]*[\w]*:)([\w]*)","")


Add datetime objects

In [118]:
def add_datetime(df_weather,current_day):
  '''
  Add datetime objects to the day-night pairs
  '''
  track_period = 0
  dict_weather = df_weather.to_dict('records')
  dict_weather_date = []
  for period in range(len(dict_weather)):
    
    # Check if first set
    if period == 0:

      dict_weather_date.append({
        'date': (datetime.now() + timedelta(days=(track_period))),
        'period': dict_weather[period]['period'],
        'short_desc': dict_weather[period]['short_desc'],
        'desc': dict_weather[period]['desc'],
        'temp_fahrenheit': dict_weather[period]['temp_fahrenheit']
      })

      dict_weather_date.append({
        'date': (datetime.now() + timedelta(days=(track_period))),
        'period': dict_weather[(period+1)]['period'],
        'short_desc': dict_weather[(period+1)]['short_desc'],
        'desc': dict_weather[(period+1)]['desc'],
        'temp_fahrenheit': dict_weather[(period+1)]['temp_fahrenheit']
      })

      # Update track_period
      track_period = track_period+1

    else:

      # Check if last period
      if period == (len(dict_weather)-1):

        dict_weather_date.append({
          'date': (datetime.now() + timedelta(days=(track_period))),
          'period': dict_weather[period]['period'],
          'short_desc': dict_weather[period]['short_desc'],
          'desc': dict_weather[period]['desc'],
          'temp_fahrenheit': dict_weather[period]['temp_fahrenheit']
        })
        
      else:
        # Check if even
        if period % 2 == 0:

          dict_weather_date.append({
            'date': (datetime.now() + timedelta(days=(track_period))),
            'period': dict_weather[period]['period'],
            'short_desc': dict_weather[period]['short_desc'],
            'desc': dict_weather[period]['desc'],
            'temp_fahrenheit': dict_weather[period]['temp_fahrenheit']
          })

          dict_weather_date.append({
            'date': (datetime.now() + timedelta(days=(track_period))),
            'period': dict_weather[(period+1)]['period'],
            'short_desc': dict_weather[(period+1)]['short_desc'],
            'desc': dict_weather[(period+1)]['desc'],
            'temp_fahrenheit': dict_weather[(period+1)]['temp_fahrenheit']
          })

          # Update track_period
          track_period = track_period+1

  df_weather_date = pd.DataFrame(dict_weather_date)

  return df_weather_date

In [119]:
current_day = datetime.now()

df_weather_datetime = add_datetime(weather, current_day)

df_weather_datetime

,date,period,short_desc,desc,temp_fahrenheit
0,2022-11-11 09:37:59.456756,VeteransDay,Heavy Rain,"Showers, with thunderstorms also possible aft...",69
1,2022-11-11 09:37:59.456760,Tonight,Showers thenSlight ChanceShowers,Showers and possibly a thunderstorm before mi...,53
2,2022-11-12 09:37:59.456762,Saturday,ChanceShowers,"A chance of showers, mainly after 11am. Part...",57
3,2022-11-12 09:37:59.456763,SaturdayNight,Slight ChanceShowers,A slight chance of showers before 1am. Mostl...,32
4,2022-11-13 09:37:59.456765,Sunday,Mostly Sunny,"Mostly sunny, with a high near 41. Northwest ...",41
5,2022-11-13 09:37:59.456766,SundayNight,Mostly Clear,"Mostly clear, with a low around 24.",24
6,2022-11-14 09:37:59.456767,Monday,Mostly Sunny,"Mostly sunny, with a high near 45.",45
7,2022-11-14 09:37:59.456768,MondayNight,Partly Cloudy,"Partly cloudy, with a low around 26.",26
8,2022-11-15 09:37:59.456769,Tuesday,ChanceRain/Snowthen ChanceRain,"A chance of snow before 10am, then a chance o...",41


In [120]:
# Output as CSV
df_weather_datetime.to_csv('./weather-gov-seven-day-forecast.csv', index=False)